In [1]:
from model.transformer_MT import *
from model.sentencepiece import *
from NLU.data_process import *
from NLU.utils import *
from NLU.train import *
from NLU.inference import *
from NLU.NLU_model import *
from BERT.vocab import *
# import warnings
# warnings.filterwarnings("ignore")

In [2]:
TRAIN = False
if TRAIN:
    loc1 = os.path.join(constant.BERT_data_path, 'ko.txt')
    loc2 = os.path.join(constant.BERT_data_path, 'sum.txt')
    loc1_num = sum(1 for line in open(loc1))
    loc2_num = sum(1 for line in open(loc2))
    print(loc1_num)
    print(loc2_num)
    up_sample = int(loc1_num / loc2_num) if int(loc1_num / loc2_num) > 1 else 1
    filenames = [loc1, loc2]
    fileiter = [1, up_sample]
    sp, vocabs = build_sp(is_train=True, filenames=filenames, fileiter=fileiter)
    #sp, vocabs = build_sp(is_train=True, filenames=sc)
else:
    sp, vocabs = build_sp(is_train=False)

In [3]:
TRAIN = False
if TRAIN:
    vocab = WordVocab(vocabs)
    save_path = os.path.join(constant.BERT_model_path, 'vocab.pkl')
    vocab.save_vocab(save_path)
else:
    save_path = os.path.join(constant.BERT_model_path, 'vocab.pkl')
    vocab = WordVocab.load_vocab(save_path)

In [4]:
len(vocab.stoi)

10004

In [5]:
DL = Multi_Turn_Processor(constant.NLU_data_path, vocab.stoi)

In [6]:
print(DL.get_params())
print(constant.n_utter)
print(DL.num_train,DL.num_valid,DL.num_test)

(10004, 269, 42, 0)
5
768 76 10


In [7]:
n_utter=constant.n_utter
save_path = os.path.join(constant.NLU_model_path, 'N'+str(n_utter)+'.pth')

In [8]:
transformer = MT_Transformer(len(vocab.itos)).to(constant.device)
model = TFNLU(transformer, len(DL.intent_to_idx), len(DL.entity_to_idx), vocab.pad_index, 
              cuda=True, 
              load_weight=False).to(constant.device)

/root/pcy/For_Service/NLU/NLU_model.py:28: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


In [9]:
crit = nn.NLLLoss(reduction='none')
adam = torch.optim.Adam(model.parameters(), lr=0, weight_decay=1e-3, betas=(0.9, 0.999), eps=1e-6)
opt = NoamOpt(constant.d_model, 1, constant.warmup_step, adam)
trainer = TrainModule(model, DL, sp, crit, opt)

In [10]:
print(save_path)

./NLU/saved_model/N5.pth


In [11]:
#Without BERT pretraining
trainer.do_train(save_path, int_coef=1.0, ent_coef=1.0, patience=15, max_epoch=1000)

100%|██████████| 3/3 [01:12<00:00, 23.58s/it]
[0]Train| E_loss:23.603880 | I_loss:4.860927 | E_F1:0.444403 | I_F1:0.013526
[0]Valid| E_loss:11.255936 | I_loss:3.809854 | E_F1:0.803328 | I_F1:0.017123
100%|██████████| 3/3 [01:09<00:00, 22.57s/it]
[1]Train| E_loss:11.257338 | I_loss:3.670179 | E_F1:0.788147 | I_F1:0.108839
[1]Valid| E_loss:8.279479 | I_loss:3.343463 | E_F1:0.803328 | I_F1:0.152397
100%|██████████| 3/3 [01:10<00:00, 23.59s/it]
[2]Train| E_loss:9.510267 | I_loss:3.515958 | E_F1:0.788226 | I_F1:0.089000
[2]Valid| E_loss:8.183312 | I_loss:3.375053 | E_F1:0.803328 | I_F1:0.179795
100%|██████████| 3/3 [01:14<00:00, 24.51s/it]
[3]Train| E_loss:8.770456 | I_loss:3.522423 | E_F1:0.788063 | I_F1:0.086212
[3]Valid| E_loss:6.843113 | I_loss:3.278428 | E_F1:0.803328 | I_F1:0.174658
100%|██████████| 3/3 [01:15<00:00, 24.25s/it]
[4]Train| E_loss:8.191815 | I_loss:3.352535 | E_F1:0.780555 | I_F1:0.157659
[4]Valid| E_loss:6.785984 | I_loss:3.531600 | E_F1:0.803873 | I_F1:0.027397
100%|██

In [14]:
#With BERT pretraining
trainer.do_train(save_path, int_coef=1.0, ent_coef=1.0, patience=15, max_epoch=1000)

100%|██████████| 3/3 [01:11<00:00, 23.10s/it]
[0]Train| E_loss:58.994197 | I_loss:5.299062 | E_F1:0.004190 | I_F1:0.011507
[0]Valid| E_loss:50.147120 | I_loss:4.954790 | E_F1:0.009002 | I_F1:0.013699
100%|██████████| 3/3 [01:15<00:00, 26.12s/it]
[1]Train| E_loss:46.195791 | I_loss:4.759369 | E_F1:0.046319 | I_F1:0.014870
[1]Valid| E_loss:30.832137 | I_loss:4.259275 | E_F1:0.183579 | I_F1:0.027397
100%|██████████| 3/3 [01:14<00:00, 24.30s/it]
[2]Train| E_loss:26.225343 | I_loss:4.344827 | E_F1:0.375085 | I_F1:0.051837
[2]Valid| E_loss:16.381251 | I_loss:3.929570 | E_F1:0.691217 | I_F1:0.075342
100%|██████████| 3/3 [01:18<00:00, 25.71s/it]
[3]Train| E_loss:17.175539 | I_loss:3.967480 | E_F1:0.700667 | I_F1:0.089793
[3]Valid| E_loss:13.277572 | I_loss:3.553801 | E_F1:0.762139 | I_F1:0.092466
100%|██████████| 3/3 [01:14<00:00, 23.80s/it]
[4]Train| E_loss:14.334000 | I_loss:3.573433 | E_F1:0.733329 | I_F1:0.109323
[4]Valid| E_loss:10.747901 | I_loss:3.338013 | E_F1:0.774414 | I_F1:0.157534
